In [1]:
import glob
import os
import re
import collections
import nltk
import random
import pylab as pl
import numpy as np
import operator
import matplotlib.pyplot as plt

dic_loc = 'C:\Users\Ziyan Liu\Desktop\Cornell\Study\cs4740\data\data_corrected\spell_checking_task\\confusion_set.txt'
head = 'C:\Users\Ziyan Liu\Desktop\Cornell\Study\cs4740\data\data_corrected\spell_checking_task\\'
text_type = ['atheism', 'autos', 'graphics','medicine','motorcycles','religion','space']
tail = '\\train_docs\*.txt'
test_head = 'C:\Users\Ziyan Liu\Desktop\Cornell\Study\cs4740\data\data_corrected\spell_checking_task\\'
test_tail = '\\test_modified_docs\*.txt'
test_head = 'C:\Users\Ziyan Liu\Desktop\Cornell\Study\cs4740\data\data_corrected\spell_checking_task\\'
test_tail = '\\test_modified_docs\*.txt'
output_head = 'C:\Users\Ziyan Liu\Desktop\Cornell\Study\cs4740\data\data_corrected\spell_checking_task\\'
output_tail = '\\train_genr_docs\\'
k = 2 # threshold for ukw

# read pairs of words that we need to distinguish
def read_target_word(dic_loc):
    data1 = []
    data2 = []

    f = open(dic_loc, 'r')
    for line in f:
        temp = line.split()
        data1.append(temp[0])
        data2.append(temp[1])
    # # needed?
    data1[0] = 'went'

    return data1, data2, data1 + data2

def txt_clean(filepath):
    #first set up some string to cut off the head of the e-mail
    # headStr1 = 'writes :'
    # headStr2 = 'wrote :'
    # headStr3 = 'said :'
    # headStr4 = 'Subject : Re : '
    # headStr5 = 'Subject : '

    #this regular expression is set to capture the email address
    regex = re.compile(("([a-z0-9!#$%&'*+\/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+\/=?^_`"
                        "{|}~-]+)*(@|\sat\s)(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?(\.|"
                        "\sdot\s))+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?)"))

    #this is to set a regular expression which will be used to capture the first occurance of letter
    r_head = re.compile("([a-zA-Z]+?)")

    # set up Text
    Text = ''

    #read all the file now
    files = glob.glob(filepath)
    for file in files:

        Text_Aggregate = ''
        f = open(file, 'r')
        line = f.read().replace('\n', '').lower()

        # # leave out head (Subject, Email Address, etc)
        # if line.rfind(headStr1) != -1:
        #     ind = line.rfind(headStr1)
        #     data = line[(ind+len(headStr1)):]
        # elif line.rfind(headStr2) != -1:
        #     ind = line.rfind(headStr2)
        #     data = line[(ind+len(headStr2)):]
        # elif line.rfind(headStr3) != -1:
        #     ind = line.rfind(headStr3)
        #     data = line[(ind + len(headStr3)):]
        # elif line.rfind(headStr4) != -1:
        #     ind = line.rfind(headStr4)
        #     data = line[(ind + len(headStr4)):]
        # elif line.rfind(headStr5) != -1:
        #     ind = line.rfind(headStr5)
        #     data = line[(ind + len(headStr5)):]
        # else:
        #     data = line
        data = line

        #clean the symbol >
        data_clear_symbol = re.sub('[>]', '', data)

        #this is to clean the signture after - - -
        idx = data_clear_symbol.find('- - -')
        if (data_clear_symbol.find('- - -') != -1 and len(data_clear_symbol[idx+1:]) <150):
            data_c1 = data_clear_symbol[0:idx+1]
        else:
            data_c1 = data_clear_symbol

        #this is to clean the signture after - -
        idx2 = data_c1.find('- -')
        if (data_c1.find('- -') != -1 and len(data_c1[idx2+1:]) <150):
            data_clear_sig = data_c1[0:idx2+1]
        else:
            data_clear_sig = data_c1

        #delete all the email address
        for email in re.findall(regex, data_clear_sig):
            data_clear_sig = data_clear_sig.replace(email[0],'')

        #replace all the " ' " to space
        data_clear_sig = data_clear_sig.replace(" ' ",'')

        # replace uneccesary notation
        rmList = '> " | # : - ) ( * [ ] } { + = ^ __ ~ / \\'
        rmList = rmList.split()
        for n in rmList:
            data_clear_sig = data_clear_sig.replace(n, '')

        # switch multiple blanks into single ones
        data_postclean = ' '.join(data_clear_sig.split())

        #let all the string tart from letter and end with letter
        idx_head = re.search(r_head, data_postclean)
        idx_gethead = idx_head.start()
        data_after = data_postclean[idx_gethead:]

        #print file, '\n', data_after, '\n'
        data_after = data_after.strip()

        #replace all the ...
        sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
        sen_list = ['<s> ' + s for s in sent_detector.tokenize(data_after.strip())]
        sen_str = ' '.join(sen_list)

        #get a string for all the email in the folder
        Text = Text + ' ' + sen_str
        Text = Text + ' <s> '
        Text_Aggregate = Text_Aggregate + Text

    Data = Text_Aggregate.split(' ')
    return Data

# decide which words should be UNK, replace those in train set
def Set_Unknown(TextList, k, target_list):
    ukw_list = []

    wd_base = list(set(TextList))
    wd_count = collections.Counter(TextList)

    for wd in wd_base:
        if ( wd not in target_list ) & ( wd_count[wd] < k ):
            ukw_list.append(wd)
            TextList = ['UKW' if x == wd else x for x in TextList]

    return ukw_list, TextList

# train bigram model
def train_bigram(target_list, TextList):

    wd_base = list(set(TextList))
    wd_count = collections.Counter(TextList)

    BiGram = {}
    for wd in wd_base:
        BiGram[wd] = {}

    for i in range(len(TextList) - 1):
        BiGram[TextList[i]][TextList[i + 1]] = 0

    for i in range(len(TextList) - 1):
        wd = TextList[i]
        wd1 = TextList[i+1]
        BiGram[wd][wd1] = BiGram[wd][wd1] + 1

    for Key in BiGram.keys():
        for key, value in BiGram[Key].items():
            BiGram[Key][key] = 1. * value / wd_count[Key]

    # BiGram = {}
    # for wd in wd_base:
    #     BiGram[wd] = {}
    #
    # for i in range(len(TextList) - 1):
    #     wd = TextList[i]
    #     wd1 = TextList[i+1]
    #     if wd1 in target_list:
    #         if wd1 not in BiGram[wd]:
    #             BiGram[wd][wd1] = 0
    #         else:
    #             BiGram[wd][wd1] = BiGram[wd][wd1] + 1
    #
    # for Key in BiGram.keys():
    #     for key, value in BiGram[Key].items():
    #         BiGram[Key][key] = 1. * value / wd_count[Key]

    return wd_base, BiGram

# check and change the word if necessary
#def check_word(ukw_list, TestList, target_list1, target_list2, target_list, Bigram):
def check_word(TestList, target_list1, target_list2, target_list, Bigram):
    time = 0

    for i in range(len(TestList) - 1):
        if TestList[i+1] in target_list1:
            p1 = 0
            p2 = 0
            cwd = target_list2[target_list1.index(TestList[i+1])]

            if TestList[i+1] in Bigram[TestList[i]].keys():
                p1 = Bigram[TestList[i]][TestList[i+1]]

            if cwd in Bigram[TestList[i]].keys():
                p2 = Bigram[TestList[i]][cwd]

            if p1 < p2:
                TestList[i+1] = cwd
                time = time + 1
                # print 'p1 = ', p1
                # print 'p2 = ', p2
                continue

        if TestList[i+1] in target_list2:
            p1 = 0
            p2 = 0
            cwd = target_list1[target_list2.index(TestList[i+1])]

            if TestList[i+1] in Bigram[TestList[i]].keys():
                p2 = Bigram[TestList[i]][TestList[i+1]]

            if cwd in Bigram[TestList[i]].keys():
                p1 = Bigram[TestList[i]][cwd]

            if p1 > p2:
                TestList[i+1] = cwd
                time = time + 1
                # print 'p1 = ', p1
                # print 'p2 = ', p2
                continue

    return time, TestList

# def main():

#     target_list = read_target_word(dic_loc);

#     for t in text_type:
#         # get list of target words
#         target_list1, target_list2, target_list = read_target_word(dic_loc)

#         # clean train texts
#         filepath = head + t + tail
#         data = txt_clean(filepath)

#         # get unw vocabulary based on train set
#         ukw_list, data = Set_Unknown(data, k, target_list)

#         # get vocabulary and train biGram
#         wd_base, Bigram = train_bigram(target_list, data)

#         # get the the files to be revised in test set
#         text_filepath = test_head + t + test_tail
#         test_files = glob.glob(text_filepath)

#         # for each of the file to be revised
#         for test_file in test_files:

#             # read file
#             f = open(test_file, 'r')
#             line = f.read().replace('\n', '').lower()
#             data_test = line.split(' ')

#             # assign unknow words
#             temp2 = set(wd_base)
#             temp2 = list(temp2.difference(set(ukw_list)))
#             data_test = ['UKW' if x not in temp2 else x for x in data_test]

#             # change the word if necessary
#             time, TestList = check_word(data_test, target_list1, target_list2, target_list, Bigram)
#             print time

#             # output files
#             output_file_name = output_head + t + output_tail + test_file[test_file.find(t + '_'):-12] + 'output.txt'

#             text_file = open(output_file_name, "w")
#             text_file.write(' '.join(TestList))
#             text_file.close()

#             for ii in test_files:
#                 print ii


#             break
#         break

# main()

In [3]:
def main():

    target_list = read_target_word(dic_loc);

    for t in text_type:
        # get list of target words
        target_list1, target_list2, target_list = read_target_word(dic_loc)

        # clean train texts
        filepath = head + t + tail
        data = txt_clean(filepath)

        # get unw vocabulary based on train set
        ukw_list, data = Set_Unknown(data, k, target_list)

        # get vocabulary and train biGram
        wd_base, Bigram = train_bigram(target_list, data)

        # get the the files to be revised in test set
        text_filepath = test_head + t + test_tail
        test_files = glob.glob(text_filepath)

        # for each of the file to be revised
        for test_file in test_files:

            # read file
            f = open(test_file, 'r')
            line = f.read().replace('\n', '').lower()
            data_test = line.split(' ')

            # assign unknow words
            temp2 = set(wd_base)
            temp2 = list(temp2.difference(set(ukw_list)))
            data_test = ['UKW' if x not in temp2 else x for x in data_test]

            # change the word if necessary
            time, TestList = check_word(data_test, target_list1, target_list2, target_list, Bigram)
            print time

            # output files
            output_file_name = output_head + t + output_tail + test_file[test_file.find(t + '_'):-12] + 'output.txt'

            text_file = open(output_file_name, "w")
            text_file.write(' '.join(TestList))
            text_file.close()

            for ii in test_files:
                print ii


            break
        break

main()

35
C:\Users\Ziyan Liu\Desktop\Cornell\Study\cs4740\data\data_corrected\spell_checking_task\atheism\test_modified_docs\atheism_file0_modified.txt
C:\Users\Ziyan Liu\Desktop\Cornell\Study\cs4740\data\data_corrected\spell_checking_task\atheism\test_modified_docs\atheism_file10_modified.txt
C:\Users\Ziyan Liu\Desktop\Cornell\Study\cs4740\data\data_corrected\spell_checking_task\atheism\test_modified_docs\atheism_file11_modified.txt
C:\Users\Ziyan Liu\Desktop\Cornell\Study\cs4740\data\data_corrected\spell_checking_task\atheism\test_modified_docs\atheism_file12_modified.txt
C:\Users\Ziyan Liu\Desktop\Cornell\Study\cs4740\data\data_corrected\spell_checking_task\atheism\test_modified_docs\atheism_file13_modified.txt
C:\Users\Ziyan Liu\Desktop\Cornell\Study\cs4740\data\data_corrected\spell_checking_task\atheism\test_modified_docs\atheism_file14_modified.txt
C:\Users\Ziyan Liu\Desktop\Cornell\Study\cs4740\data\data_corrected\spell_checking_task\atheism\test_modified_docs\atheism_file15_modified.t